# CLIENT

In [ ]:
import phe as paillier
import json

In [ ]:
def storeKeys():
    public_key, private_key = paillier.generate_paillier_keypair()
    keys = {}
    keys['public_key'] = {'n': public_key.n}
    keys['private_key'] = {'p': private_key.p, 'q': private_key.q}
    with open('keys/client_keys.json', 'w') as file:
        json.dump(keys, file)


def getKeys():
    with open('keys/client_keys.json', 'r') as file:
        keys = json.load(file)
        pub_key = paillier.PaillierPublicKey(n=int(keys['public_key']['n']))
        priv_key = paillier.PaillierPrivateKey(
            pub_key, keys['private_key']['p'], keys['private_key']['q'])
        return pub_key, priv_key


def serializeData(public_key, data):
    encrypted_data_list = [public_key.encrypt(x) for x in data]
    encrypted_data = {}
    encrypted_data['public_key'] = {'n': public_key.n}
    encrypted_data['values'] = [(str(x.ciphertext()), x.exponent)
                                for x in encrypted_data_list]
    serialized = json.dumps(encrypted_data)
    return serialized

### Create SK & PK

In [ ]:
storeKeys()

### Data Encryption

In [ ]:
pub_key, priv_key = getKeys()
# age,healthy_eating,active_lifestyle,Gender
data = age, he, al, gen = [24, 4, 6, 1]
serializeData(pub_key, data)
datafile = serializeData(pub_key, data)
with open('keys/encrypted_data.json', 'w') as file:
    json.dump(datafile, file)

### Send Computation to serverless
Skip the traditional file data transferring

### Data Decryption

In [ ]:
def loadComputation():
    with open('keys/cyphered_computation.json', 'r') as file: 
        ans=json.load(file)
        answer=json.loads(ans)
        return answer

In [ ]:
comp_file=loadComputation()
comp_key=paillier.PaillierPublicKey(n=int(comp_file['pubkey']['n']))
comp = paillier.EncryptedNumber(comp_key, int(comp_file['values'][0]), int(comp_file['values'][1]))
if (comp_key==pub_key):
    print(priv_key.decrypt(comp))